In [1]:
from __future__ import absolute_import, division, print_function, unicode_literals

In [2]:
!pip uninstall tensorflow

Uninstalling tensorflow-2.3.0:
  Would remove:
    /usr/local/bin/estimator_ckpt_converter
    /usr/local/bin/saved_model_cli
    /usr/local/bin/tensorboard
    /usr/local/bin/tf_upgrade_v2
    /usr/local/bin/tflite_convert
    /usr/local/bin/toco
    /usr/local/bin/toco_from_protos
    /usr/local/lib/python3.6/dist-packages/tensorflow-2.3.0.dist-info/*
    /usr/local/lib/python3.6/dist-packages/tensorflow/*
Proceed (y/n)? y
  Successfully uninstalled tensorflow-2.3.0


In [3]:
!pip install tensorflow==2.0.0-beta1

     |████████████████████████████████| 87.9MB 33kB/s 
     |████████████████████████████████| 51kB 7.2MB/s 
     |████████████████████████████████| 3.1MB 44.1MB/s 
     |████████████████████████████████| 501kB 38.0MB/s 


In [4]:
import pandas as pd
import numpy as np
import nltk

df = pd.read_csv("https://raw.githubusercontent.com/huynhthanh98/AML/main/lab-04/spam_detection.csv")
df

,Text,Label,y
0,"Go until jurong point, crazy.. Available only ...",ham,0
1,Ok lar... Joking wif u oni...,ham,0
2,Free entry in 2 a wkly comp to win FA Cup fina...,spam,1
3,U dun say so early hor... U c already then say...,ham,0
4,"Nah I don't think he goes to usf, he lives aro...",ham,0
...,...,...,...
5569,This is the 2nd time we have tried 2 contact u...,spam,1
5570,Will Ã¼ b going to esplanade fr home?,ham,0
5571,"Pity, * was in mood for that. So...any other s...",ham,0
5572,The guy did some bitching but I acted like i'd...,ham,0


In [5]:
nltk.download('punkt')
texts = df["Text"].to_list()
texts = [text.lower() for text in texts ]           # chuyển các đoạn text thành chữ thường (word embedding chỉ cho chữ thường)
tokenized_texts = [nltk.tokenize.word_tokenize(text) for text in texts]    # tách câu thành một list các từ

print(tokenized_texts[0])

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
['go', 'until', 'jurong', 'point', ',', 'crazy..', 'available', 'only', 'in', 'bugis', 'n', 'great', 'world', 'la', 'e', 'buffet', '...', 'cine', 'there', 'got', 'amore', 'wat', '...']


In [6]:
!wget http://nlp.stanford.edu/data/glove.6B.zip

--2020-11-17 08:20:45--  http://nlp.stanford.edu/data/glove.6B.zip
Resolving nlp.stanford.edu (nlp.stanford.edu)... 171.64.67.140
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:80... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://nlp.stanford.edu/data/glove.6B.zip [following]
--2020-11-17 08:20:45--  https://nlp.stanford.edu/data/glove.6B.zip
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: http://downloads.cs.stanford.edu/nlp/data/glove.6B.zip [following]
--2020-11-17 08:20:46--  http://downloads.cs.stanford.edu/nlp/data/glove.6B.zip
Resolving downloads.cs.stanford.edu (downloads.cs.stanford.edu)... 171.64.64.22
Connecting to downloads.cs.stanford.edu (downloads.cs.stanford.edu)|171.64.64.22|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 862182613 (822M) [application/zip]
Saving to: ‘glove.6B.zip’

glove.6

In [7]:
! unzip glove.6B.zip

Archive:  glove.6B.zip
  inflating: glove.6B.50d.txt        
  inflating: glove.6B.100d.txt       
  inflating: glove.6B.200d.txt       
  inflating: glove.6B.300d.txt       


In [8]:
import io
import numpy as np
def load_word_embeddings(fname):
    fin = io.open(fname, 'r', encoding='utf-8', newline='\n', errors='ignore')
    vocab, matrix = [], []
    i=0
    for line in fin:
        tokens = line.rstrip().split(' ')
        vocab.append(tokens[0])
        matrix.append(list(map(float, tokens[1:])))
    return vocab, np.asarray(matrix)

In [9]:
vocab, matrix = load_word_embeddings("glove.6B.300d.txt")
matrix

array([[ 0.04656  ,  0.21318  , -0.0074364, ...,  0.0090611, -0.20989  ,
         0.053913 ],
       [-0.25539  , -0.25723  ,  0.13169  , ..., -0.2329   , -0.12226  ,
         0.35499  ],
       [-0.12559  ,  0.01363  ,  0.10306  , ..., -0.34224  , -0.022394 ,
         0.13684  ],
       ...,
       [ 0.075713 , -0.040502 ,  0.18345  , ...,  0.21838  ,  0.30967  ,
         0.43761  ],
       [ 0.81451  , -0.36221  ,  0.31186  , ...,  0.075486 ,  0.28408  ,
        -0.17559  ],
       [ 0.429191 , -0.296897 ,  0.15011  , ...,  0.28975  ,  0.32618  ,
        -0.0590532]])

In [10]:
## Gán các mã
__PADDED_INDEX__ = 0    # mã dùng cho các vị trí chỉ có tính nối dài cho cùng kích thước
__UNKNOWN_WORD__ = 1    # mã cho những từ không có trong embedding

In [11]:
# Tạo một dictionary, có nhiệm vụ là một ánh xạ từ ảnh sang mã số, mã số được bắt đầu từ 2 vì số 0 và 1 được dành cho trường hợp đặc biệt
word_to_index = {word: index+2 for index, word in enumerate(vocab)}

In [12]:
# Do do mã số được bắt đầu từ 2, nên cần thêm 2 vector vào đàu ma trận
embedding_matrix = np.pad(matrix, [[2,0],[0,0]], mode='constant', constant_values =0.0)
print(embedding_matrix)

# Khi đó, __PADDED_INDEX__ dùng dòng đầu tiên của  embedding_matrix
# __UNKNOWN_WORD__ dùng dòng thứ hai của embedding_matrix

[[ 0.         0.         0.        ...  0.         0.         0.       ]
 [ 0.         0.         0.        ...  0.         0.         0.       ]
 [ 0.04656    0.21318   -0.0074364 ...  0.0090611 -0.20989    0.053913 ]
 ...
 [ 0.075713  -0.040502   0.18345   ...  0.21838    0.30967    0.43761  ]
 [ 0.81451   -0.36221    0.31186   ...  0.075486   0.28408   -0.17559  ]
 [ 0.429191  -0.296897   0.15011   ...  0.28975    0.32618   -0.0590532]]


In [13]:
## Bây giờ ta sẽ chuyển data spam dection thành các mã số
import tensorflow as tf

X = []
for text in tokenized_texts:
    cur_text_indices = []
    for word in text:
        if word in word_to_index:
            cur_text_indices.append(word_to_index[word])    ## map từ word sang index
        else:
            cur_text_indices.append(__UNKNOWN_WORD__)       ## gán unknown cho từ không có trong bộ glove
    X.append(cur_text_indices)

## pad data cho có cùng chiều dài
X = tf.keras.preprocessing.sequence.pad_sequences(sequences=X,       # sequences: list các câu có độ dài không bằng nhau
                                                  padding='post')    # vị trí pad là 'pre' (trước) hoặc 'post' (sau)

y = df['y'].values   ## Label của bài toán, 0 là không phải spam, 1 là spam

/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:519: FutureWarning: Passing (type, 1) or 

In [14]:
from sklearn.model_selection import train_test_split
X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size= 0.2, random_state =0)

**Mô hình**

In [15]:
from tensorflow.keras.layers import Input, Embedding, LSTM, Bidirectional, Dense
from tensorflow.keras.models import Model

inputs = Input(shape=(None,))                   ## None biểu thị kích thước không xác định của câu

embed = Embedding(input_dim=embedding_matrix.shape[0],   ## Khai báo kích thước của vocab
                 output_dim=embedding_matrix.shape[1],   ## Khai báo kích thước của embedding
                  embeddings_initializer = tf.keras.initializers.Constant(value=embedding_matrix),  ## Khởi tạo cho embedding bằng ma trận có sẵn
                  trainable=False,                       ## Không cần thiết train embedding
                 mask_zero=True)(inputs)                 ## zero_mask: những vị trí có giá trị 0 không được tính toán, vì đó là giá trị thêm vào cho đủ độ dài mà thôi
                                                         ##  (__PADDED_INDEX__ gán bằng 0)

lstm = LSTM(units=100,                          ## units: kích thước của hidden_state trong LSTM
            return_sequences=False)(embed)      ## return_sequences: LSTM trả về toàn bộ  hay là trả về hidden_state cuối cùng

dense = Dense(units=2, activation='softmax')(lstm)
model = Model(inputs=inputs,
              outputs=dense)

model.compile(optimizer='adam',
             loss='sparse_categorical_crossentropy',
             metrics=['accuracy'])
model.summary()

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, None)]            0         
_________________________________________________________________
embedding (Embedding)        (None, None, 300)         120000600 
_________________________________________________________________
lstm (LSTM)                  (None, 100)               160400    
_________________________________________________________________
dense (Dense)                (None, 2)                 202       
Total params: 120,161,202
Trainable params: 160,602
Non-trainable params: 120,000,600
_________________________________________________________________


In [16]:
mc = tf.keras.callbacks.ModelCheckpoint(filepath="lstm_spam.h5", 
                                     monitor='val_loss',
                                     mode='min', 
                                     verbose=0, 
                                     save_best_only=True)
# Train
model.fit(X_train, y_train, 
          validation_data=(X_valid, y_valid),
         epochs=10,
         callbacks=[mc])

model.load_weights("lstm_spam.h5")
_, val_acc = model.evaluate(X_valid, y_valid)
print("Accuracy on valid: ", val_acc)

Train on 4459 samples, validate on 1115 samples
Epoch 1/10
4459/4459 [==============================] - 204s 46ms/sample - loss: 0.1455 - accuracy: 0.9442 - val_loss: 0.0701 - val_accuracy: 0.9767
Epoch 2/10
4459/4459 [==============================] - 202s 45ms/sample - loss: 0.0578 - accuracy: 0.9830 - val_loss: 0.0650 - val_accuracy: 0.9803
Epoch 3/10
4459/4459 [==============================] - 207s 46ms/sample - loss: 0.0444 - accuracy: 0.9870 - val_loss: 0.0580 - val_accuracy: 0.9821
Epoch 4/10
4459/4459 [==============================] - 212s 48ms/sample - loss: 0.0347 - accuracy: 0.9892 - val_loss: 0.0536 - val_accuracy: 0.9857
Epoch 5/10
4459/4459 [==============================] - 101s 23ms/sample - loss: 0.0224 - accuracy: 0.9939 - val_loss: 0.0550 - val_accuracy: 0.9839
Epoch 6/10
4459/4459 [==============================] - 207s 46ms/sample - loss: 0.0126 - accuracy: 0.9978 - val_loss: 0.0523 - val_accuracy: 0.9874
Epoch 7/10
4459/4459 [==============================] - 10

In [64]:
def compare(arr):
  spam = True
  if (arr[0][0] > arr[0][1]):
    spam = False
  return spam

In [66]:
def model_predict(text):
  text_test = [text.lower()]           # chuyển các đoạn text thành chữ thường 
  tokenized_text_test = [nltk.tokenize.word_tokenize(text)]    # tách câu thành một list các từ

  X_test = []
  for text in tokenized_text_test:
      cur_text_indices = []
      for word in text:
          if word in word_to_index:
              cur_text_indices.append(word_to_index[word])    ## map từ word sang index
          else:
              cur_text_indices.append(__UNKNOWN_WORD__)       
      X_test.append(cur_text_indices)

  ## pad data cho có cùng chiều dài
  X_test = tf.keras.preprocessing.sequence.pad_sequences(sequences=X_test,       # sequences: list các câu có độ dài không bằng nhau
                                                    padding='post')    # vị trí pad là 'pre' (trước) hoặc 'post' (sau)
  #predict
  return compare(model.predict(X_test, verbose=0))                                                  

In [68]:
text_test = []
text_test.append("wanna ask something? just send me a mess")
text_test.append("Urgent! You have won our competition's prize!! Please call us now.")
text_test.append("Call me to get a free holiday now")

for i in range(len(text_test)):
  print(text_test[i])
  if (model_predict(text_test[i]) == True):
    print("This is spam!")
  else:
    print("This is not spam")

wanna ask something? just send me a mess
This is not spam
Urgent! You have won our competition's prize!! Please call us now.
This is spam!
Call me to get a free holiday now
This is not spam
